## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件 对于提供的数据集，不要改存储地方，也不要修改文件名和内容 不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块 写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！ 
这次作业很重要，一定要完成！ 相信会有很多的收获！

In [1]:
import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

## 加载训练集文件和测试集文件

In [2]:
data_text,data=[],[]
stopWords=[]

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv('data/train_clean.csv', sep='\t'),
        pd.read_csv('data/dev_clean.csv', sep='\t'),
        pd.read_csv('data/test_clean.csv', sep='\t')
        ])
    print("读取数据集完成")
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
    stopWords = open('data/stopwords.txt').readlines()
    print("取停用词完成")
    return data, data_text,datatext, stopWords


In [3]:
data, data_text, datatext, stopWords=load_data()

load Pre_process
读取数据集完成
取停用词完成


In [29]:
data['text'][1]

1    额吉 的 白云 优秀 蒙古文 文学作品 翻译 出版 工程 第五辑 散文 卷 散文集 额吉 的...
1    佛罗伦萨 在 哪里 佛罗伦萨 在 哪里 题意 深沉 , 作者 置身于 佛罗伦萨 却 反复 叩...
1    历代 茶 诗集 成宋 金卷 本书 主要 内容 包括 : 丁开 摘句 一首 、 丁带 茶 诗 ...
Name: text, dtype: object

In [6]:
TfidfVectorizer?

In [4]:
def trainer_tfidf():
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练tfidf的词向量
    # 第一步：首先通过TfidfVectorizer创建一个模型对象
    count_vect = TfidfVectorizer(stop_words=stopWords,max_df =0.4,min_df=0.001,ngram_range=(1,2))
    # 第二步：用模型对象去fit训练数据集
    tfidf = count_vect.fit(data['text'])
    print('train tfidf_embedding')
    #返回是一个稀疏矩阵
    return tfidf


In [5]:
tfidf = trainer_tfidf()

/Users/kaifeng/ml/env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', 'sub', 'sup', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｃ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '一一', '一个', '一些', '一切', '一则', '一方面', '一旦', '一来', '一样', '一番', '一直', '一般', '万一', '上下', '不仅', '不但', '不光', '不单', '不只', '不如', '不怕', '不惟', '不成', '不拘', '不比', '不然', '不特', '不独', '不管', '不论', '不过', '不问', '与其', '与否', '与此同时', '两者', '为了', '为什么', '为何', '为着', '乃至', '之一', '之所以', '之类', '乌乎', '也好', '也就是说', '也罢', '于是', '于是乎', '云云', '人家', '什么', '什么样', '从而', '他人', '他们', '以便', '以免', '以及', '以至', '以至于', '以致', '任何', '任凭', '似的', '但是', '何况'

train tfidf_embedding


根据数据集训练word2vec的词向量  
利用models.Word2Vec训练w2v的词向量  
部分参数说明：  
min_count：表示低于该频率的词将会删除，  
window：表示滑动窗口大小，  
alpha：表示学习率，  
negative：表示负采样样本个数，  
max_vocab_size：表示RAM中最大限制的词个数  

In [6]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
    # 训练w2v的词向量
    # 第一步：利用models.Word2Vec构建一个模型，
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000) 
        
    w2v.build_vocab(datatext)

    w2v.train(datatext,
              total_examples=w2v.corpus_count,
              epochs=15,
              report_delay=1)
        
    
    return w2v

In [7]:
w2v = trainer_w2v()

train word2vec Embedding
train fast_embedding


In [8]:
models.FastText?

In [9]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练FastTExt的词向量
    # hint: 利用models.FastText,
    # 可使用部分参数说明：
    # size：生成的向量维度，
    # window: 移动窗口，
    # aphla: 学习率，
    # min_count: 对低于该频率的词进行截断
    # 可以参照trainer_w2v函数完成FastText的词向量的训练
    # 可以直接根据models.FastText直接训练样本获取词向量
    fast = models.FastText(datatext,size=300,window=3,alpha=0.03,min_count=2,iter=30,word_ngrams=2,max_vocab_size=50000)
    return fast

In [10]:
fast= trainer_fasttext()

In [12]:
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    
    
    print('save tfidf model')
    joblib.dump(tfidf,'tfidf')

    # hint: w2v可以通过自带的save函数进行保存
    print('save word2vec model')
    w2v.wv.save_word2vec_format('w2v.bin',binary=False)
     # hint: fast可以通过自带的save函数进行保存
    print('save fast model')
    fast.wv.save_word2vec_format('fast.bin',binary=False)

    
saver()
    

save tfidf model
save word2vec model
save fast model


In [13]:
def load():
    '''
    函数说明：该函数加载训练好的模型
    '''
    #ToDo
    # 加载模型 
    # hint: tfidf可以通过joblib.load进行加载
    # w2v和fast可以通过gensim.models.KeyedVectors.load加载
    print('load tfidf_embedding model')
    tfidf =  joblib.load('tfidf')
    print('load w2v_embedding model')
    w2v =  models.KeyedVectors.load_word2vec_format('w2v.bin',binary=False)
    print('load fast_embedding model')
    fast =  models.KeyedVectors.load_word2vec_format('fast.bin',binary=False)
load()

load tfidf_embedding model
load w2v_embedding model
load fast_embedding model
